In [1]:
import wfdb
from wfdb import processing
import matplotlib.pyplot as plt
import numpy as np
from ecgdetectors import Detectors
import biosppy
import cv2
from biosppy.signals import ecg
import json

In [ ]:
#ls

In [2]:
record = wfdb.rdrecord('./100', sampfrom=0, sampto=10000, channels=[0])
qrs_inds = processing.qrs.gqrs_detect(sig=record.p_signal[:,0], fs=record.fs)
print(qrs_inds)

[ 357  650  934 1219 1502 1797 2032 2390 2693 2985 3270 3547 3850 4158
 4453 4752 5048 5334 5621 5906 6202 6514 6811 7093 7379 7657 7940 8233
 8526 8825 9129 9419 9698]


In [ ]:
def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()

In [ ]:
peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
         title="GQRS peak detection on record 100")

# QRS detection in ecg using wfdb library

In [ ]:
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.peaks.correct_peaks(record.p_signal[:,0], 
                                                     peak_inds=qrs_inds,
                                                     search_radius=search_radius, 
                                                     smooth_window_size=150)

In [ ]:
print('Corrected GQRS detected peak indices:', sorted(corrected_peak_inds))
peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/100")

In [ ]:
sig, fields = wfdb.rdsamp('./100', channels=[0], sampto=15000)
ann_ref = wfdb.rdann('./100','atr', sampto=15000)
unfiltered_ecg = sig[:, 0]  
# Run QRS detection on signal
xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
xqrs.detect()

In [ ]:
print(fields)

# Detecting the R_peaks using py-ecg-detectors 1.0.2
## The r signals are found to be present in various locations in the above plotted graph

In [ ]:
detectors = Detectors(360)
r_peaks = detectors.christov_detector(unfiltered_ecg)
print("Christov r_peaks value.....")
print(r_peaks)
print("\n")

#hamilton algorithm
r_peaks1 = detectors.hamilton_detector(unfiltered_ecg)
print("Hamilton r_peaks value.....")
print(r_peaks1)


## The below code shows us the annotated symbols that are presented in the MIT BIH database.
### The annoted values read from rdann is used to list the annotations.

In [ ]:
wfdb.show_ann_labels()

In [ ]:
record = wfdb.rdrecord('./100',
                       sampfrom=300, sampto=1000)
wfdb.plot_wfdb(record, title='Record p000878/3269321_0001') 
display(record.__dict__)

## The above code contains the details that are found in the .hea file
## The following are the explanations for the above values in the dictionary...

###  Attributes
    p_signal : ndarray
        An (MxN) 2d numpy array, where M is the signal length. Gives the
        physical signal values intended to be written. Either p_signal or
        d_signal must be set, but not both. If p_signal is set, this method will
        use it to perform analogue-digital conversion, writing the resultant
        digital values to the dat file(s). If fmt is set, gain and baseline must
        be set or unset together. If fmt is unset, gain and baseline must both
        be unset.
    d_signal : ndarray
        An (MxN) 2d numpy array, where M is the signal length. Gives the
        digital signal values intended to be directly written to the dat
        file(s). The dtype must be an integer type. Either p_signal or d_signal
        must be set, but not both. In addition, if d_signal is set, fmt, gain
        and baseline must also all be set.
    p_signal : ndarray
        The expanded physical conversion of the signal. Either a 2d numpy
        array or a list of 1d numpy arrays.
    e_d_signal : ndarray
        The expanded digital conversion of the signal. Either a 2d numpy
        array or a list of 1d numpy arrays.
    record_name : str
        The name of the WFDB record to be read, without any file
        extensions. If the argument contains any path delimiter
        characters, the argument will be interpreted as PATH/BASE_RECORD.
        Both relative and absolute paths are accepted. If the `pn_dir`
        parameter is set, this parameter should contain just the base
        record name, and the files fill be searched for remotely.
        Otherwise, the data files will be searched for in the local path.
    n_sig : int
        Total number of signals.
    fs : float
        The sampling frequency of the record.
    counter_freq : float
        The frequency used to start counting.
    base_counter : float
        The counter used at the start of the file.
    sig_len : int
        The total length of the signal.
    base_time : str
        A string of the record's start time in 24h 'HH:MM:SS(.ms)' format.
    base_date : str
        A string of the record's start date in 'DD/MM/YYYY' format.
    file_name : str
        The name of the file used for analysis.
    fmt : list
        A list of strings giving the WFDB format of each file used to store each
        channel. Accepted formats are: '80','212",'16','24', and '32'. There are
        other WFDB formats as specified by:
        https://www.physionet.org/physiotools/wag/signal-5.htm
        but this library will not write (though it will read) those file types.
    samps_per_frame : int
        The total number of samples per frame.
    skew : float
        The offset used to allign signals.
    byte_offset
        The byte offset used to allign signals.
    adc_gain : list
        A list of numbers specifying the ADC gain.
    baseline : list
        A list of integers specifying the digital baseline.
    units : list
        A list of strings giving the units of each signal channel.  
    adc_res: int
        The value produced by the ADC given a given Volt input.  
    adc_zero: int
        The value produced by the ADC given a 0 Volt input.
    init_value : list
        The initial value of the signal.
    checksum : list, int
        The checksum of the signal.
    block_size : str
        The dimensions of the field data.
    sig_name :
        A list of strings giving the signal name of each signal channel.
    comments : list, optional
        A list of string comments to be written to the header file.

In [ ]:
record = wfdb.rdrecord('./100')
wfdb.plot_wfdb(record=record, title='Record p000878/3269321_0001') 
display(record.__dict__)

# Can also read the same files hosted on Physionet (takes long to stream the many large files)
signals, fields = wfdb.rdsamp('100')
wfdb.plot_items(signal=signals, fs=fields['fs'], title='Record p000878/3269321_0001')
display((signals, fields))

# Biosspy

In [ ]:
out = ecg.ecg(signal=unfiltered_ecg, sampling_rate=360., show=True)

In [ ]:
def segmentation_images(data):
    path = "ECG_image"
    output = []
    flag = 1
    APC, NORMAL, LBB, PVC, PAB, RBB, VEB = [], [], [], [], [], [], []
    output.append(str(path))
    result = {"APC": APC, "Normal": NORMAL, "LBB": LBB, "PAB": PAB, "PVC": PVC, "RBB": RBB, "VEB": VEB}

        
    indices = []
        
    kernel = np.ones((4,4),np.uint8)
        
    #csv = pd.read_csv(path)
    #csv_data = csv[' Sample Value']
    #data = np.array(csv_data)
    signals = []
    count = 1
    peaks =  biosppy.signals.ecg.christov_segmenter(signal=data, sampling_rate = 200)[0]
    for i in (peaks[1:-1]):
        diff1 = abs(peaks[count - 1] - i)
        diff2 = abs(peaks[count + 1]- i)
        x = peaks[count - 1] + diff1//2
        y = peaks[count + 1] - diff2//2
        signal = data[x:y]
        signals.append(signal)
        count += 1
        indices.append((x,y))
    
            
        for count, i in enumerate(signals):
            fig = plt.figure(frameon=False)
            plt.plot(i) 
            plt.xticks([]), plt.yticks([])
            for spine in plt.gca().spines.values():
                spine.set_visible(False)

            filename = 'fig' + '.png'
            fig.savefig(filename)
            im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
            im_gray = cv2.erode(im_gray,kernel,iterations = 1)
            im_gray = cv2.resize(im_gray, (128, 128), interpolation = cv2.INTER_LANCZOS4)
            cv2.imwrite(filename, im_gray)
            im_gray = cv2.imread(filename)
            print("processing..")
            
            '''
            pred = model.predict(im_gray.reshape((1, 128, 128, 3)))
            pred_class = pred.argmax(axis=-1)
            if pred_class == 0:
                APC.append(indices[count]) 
            elif pred_class == 1:
                NORMAL.append(indices[count]) 
            elif pred_class == 2:    
                LBB.append(indices[count])
            elif pred_class == 3:
                PAB.append(indices[count])
            elif pred_class == 4:
                PVC.append(indices[count])
            elif pred_class == 5:
                RBB.append(indices[count]) 
            elif pred_class == 6:
                VEB.append(indices[count])
        '''


        result = sorted(result.items(), key = lambda y: len(y[1]))[::-1]   
        output.append(result)
        data = {}
        data['filename'+ str(flag)] = str(path)
        data['result'+str(flag)] = str(result)

        json_filename = 'data.txt'
        with open(json_filename, 'a+') as outfile:
            json.dump(data, outfile) 
        flag+=1
        
   
         
    

# Biosppy library for r_peaks

In [7]:
import wfdb
from wfdb import processing
import matplotlib.pyplot as plt
import numpy as np
from ecgdetectors import Detectors
import biosppy
import cv2
from biosppy.signals import ecg
import json
import glob


In [8]:
def get_records():
    """ Get paths for data in data/mit/ directory """
    #Download if doesn't exist
    
    # There are 3 files for each record
    # *.atr is one of them
    paths = glob.glob('./*.atr')

    # Get rid of the extension
    paths = [path[:-4] for path in paths]
    paths.sort()

    return paths

In [9]:
paths = get_records()

In [ ]:
'''directory = "./Sample_images/r_peaks"
def segmentation(records):
    signals = []
    for e in records:
        sig, fields = wfdb.rdsamp(e, channels = [0]) 
        data = sig[:, 0]
        count = 1
        peaks =  biosppy.signals.ecg.christov_segmenter(signal=data, sampling_rate = 200)[0]
        for i in (peaks[1:-1]):
            diff1 = abs(peaks[count - 1] - i)
            diff2 = abs(peaks[count + 1]- i)
            x = peaks[count - 1] + diff1//2
            y = peaks[count + 1] - diff2//2
            signal = data[x:y]
            signals.append(signal)
            count += 1
        return signals
    
def ploting(Normal):
        for count, i in enumerate(Normal):
            fig = plt.figure(frameon=False)
            plt.plot(i)
            plt.xticks([]), plt.yticks([])
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            filename = directory + '/' + str(count)+'.png'
            fig.savefig(filename)
            im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
            im_gray = cv2.resize(im_gray, (128, 128), interpolation = cv2.INTER_LANCZOS4)
            cv2.imwrite(filename, im_gray)
            
  '''  

In [19]:
directory = "./Sample_images/r_peaks"
def segmentation(records):
    Normal = []
    for e in records:
        signals, fields = wfdb.rdsamp(e, channels = [0]) 

        ann = wfdb.rdann(e, 'atr')
        good = ['N']
        ids = np.in1d(ann.symbol, good)
        imp_beats = ann.sample[ids]
        beats = (ann.sample)
        for i in imp_beats:
            beats = list(beats)
            j = beats.index(i)
            if(j!=0 and j!=(len(beats)-1)):
                x = beats[j-1]
                y = beats[j+1]
                diff1 = abs(x - beats[j])//2
                diff2 = abs(y - beats[j])//2
                Normal.append(signals[beats[j] - diff1: beats[j] + diff2, 0])
        return Normal
    
    
def r_peaks(Normal):
    data = np.array(Normal)
    signals = []
    count = 1
    peaks =  biosppy.signals.ecg.christov_segmenter(signal=data, sampling_rate = 200)[0]
    for i in (peaks[1:-1]):
        diff1 = abs(peaks[count - 1] - i)
        diff2 = abs(peaks[count + 1]- i)
        x = peaks[count - 1] + diff1//2
        y = peaks[count + 1] - diff2//2
        signal = data[x:y]
        signals.append(signal)
        count += 1
    return signals

def ploting(Normal):
        for count, i in enumerate(Normal):
            fig = plt.figure(frameon=False)
            plt.plot(i)
            plt.xticks([]), plt.yticks([])
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            filename = directory + '/' + str(count)+'.png'
            fig.savefig(filename)
            im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
            im_gray = cv2.resize(im_gray, (128, 128), interpolation = cv2.INTER_LANCZOS4)
            cv2.imwrite(filename, im_gray)
            
    

In [20]:
Normal  = segmentation(paths)